In [ ]:
import psycopg2
import os

In [ ]:
postgres_uri = os.environ.get('URI')
embedding_model_endpoint = os.environ.get("embedding_model_endpoint")

In [ ]:
def execute_sql(sql):
    try:
        conn = psycopg2.connect(postgres_uri)
        cur = conn.cursor()
        cur.execute(sql)
        conn.commit()
    finally:
        if conn is not None:
            conn.close()

In [ ]:
def execute_sql_results(sql):
    try:
        conn = psycopg2.connect(postgres_uri)
        cur = conn.cursor()
        cur.execute(sql)
        rows = cur.fetchall()
        return rows
    finally:
        if conn is not None:
            conn.close()

In [ ]:
sql = 'select film_id, title, description from film limit 10'

In [ ]:
rows = execute_sql_results(sql)
for row in rows:
    print(row)

In [ ]:
sql = """
    select film_id, title, description, 1 - (aidb.encode_text('bert', description)::vector <=>
	aidb.encode_text('bert', 'A movie about cats in Australia')::vector) as cosine_similarity_score from film order by 4 desc limit 10
    """

In [ ]:
rows = execute_sql_results(sql)
for row in rows:
    print(row)

In [ ]:
sql = f"""
    select aidb.create_model(
    'grit-llm',
    'embeddings',
    '{"model":"grit-llm", "url":{embedding_model_endpoint}, 
    "dimensions":4096}'::JSONB,
    '{"api_key":"test"}'::JSONB);
    """

In [ ]:
execute_sql(sql)

In [ ]:
sql = """
    select aidb.delete_model('grit-llm');
    """

In [ ]:
execute_sql(sql)

In [ ]:
sql = "select aidb.encode_text('grit-llm','I like it');"

In [ ]:
sql = """
    select film_id, title, description, 1 - (aidb.encode_text('grit-llm', description)::vector <=>
	aidb.encode_text('bert', 'A movie about cats in Australia')::vector) as cosine_similarity_score from film order by 4 desc limit 10
    """

In [ ]:
rows = execute_sql_results(sql)
for row in rows:
    print(row)

In [ ]:
sql = """
    select film_id, title, description, 1 - (aidb.encode_text('bert', description)::vector <=>
	aidb.encode_text('bert', 'A movie about cats in Australia')::vector) as cosine_similarity_score from film order by 4 desc limit 10
    """

In [ ]:
#Below is a curl REST example for retrieving an embedding

In [3]:
import pycurl
import json
from io import BytesIO


data = {
     "model": "grit-llm",
     "input": ["this is a test"],
     "encoding_format":"float"
}

post_data = json.dumps(data)
headers = ['Content-Type: application/json']
buffer = BytesIO()

In [4]:
c = pycurl.Curl()

c.setopt(c.URL, embedding_model_endpoint)
c.setopt(c.POSTFIELDS, post_data)
c.setopt(c.HTTPHEADER, headers)
c.setopt(c.WRITEDATA, buffer)

#c.setopt(pycurl.SSL_VERIFYPEER, 0)
#c.setopt(pycurl.SSL_VERIFYHOST, 0)
c.perform()
c.close()

response = buffer.getvalue()

result = response.decode('utf-8')
print(result[0:1000])